In [1]:
from sogaPreprocessor import *
from producecfg import *
from libSOGA import *

In [2]:
compiledFile=compile2SOGA('../programs/SOGA/SSA/Poisson.soga')

In [3]:
cfg = produce_cfg(compiledFile)

In [4]:
output_dist = start_SOGA(cfg)

In [5]:
output_dist

Dist<['y', 'x'],pi: [0.5231655423461306, 0.2747326815578447, 0.12421643039414292, 0.05017208372123393, 0.018511086668227247, 0.006332199410601337, 0.002029814542937652, 0.0006146264096662407, 0.00017689851665892638, 4.863643255636825e-05] mu: [array([0., 0.]), array([1., 0.]), array([2., 0.]), array([3., 0.]), array([4., 0.]), array([5., 0.]), array([6., 0.]), array([7., 0.]), array([8., 0.]), array([9., 0.])] sigma: [array([[0., 0.],
       [0., 0.]]), array([[0., 0.],
       [0., 0.]]), array([[0., 0.],
       [0., 0.]]), array([[0., 0.],
       [0., 0.]]), array([[0., 0.],
       [0., 0.]]), array([[0., 0.],
       [0., 0.]]), array([[0., 0.],
       [0., 0.]]), array([[0., 0.],
       [0., 0.]]), array([[0., 0.],
       [0., 0.]]), array([[0., 0.],
       [0., 0.]])]>

In [6]:
pois.pmf(0,4)

0.01831563888873418